# Librerías

In [8]:
#Librerías para subir los datos
from google.colab import files
import io

import pandas as pd

import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

import numpy as np


%matplotlib inline 

In [9]:
#Importamos modulos
from warnings import filterwarnings
filterwarnings('ignore')
import pandas as pd
import numpy as np
import io
import requests
import seaborn as sns
import timeit
import math
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import metrics

#from google.colab import files
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#Parámetros para visualización
sns.set(style='whitegrid', color_codes=True) #whitegrid #darkgrid

# Funciones

In [10]:
def upload_files (sep=','):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), sep=sep)
    # df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col='Unnamed: 0') # index sin nombre
    return df

In [11]:
def split_dataset_train_test(df_train, df_test):
  X_train, y_train=df_train.drop(['num_casos','fecha'], axis=1), df_train['num_casos']
  X_test, y_test = df_test.drop(['num_casos','fecha'], axis=1), df_test['num_casos']
  return X_train, y_train, X_test, y_test

In [12]:
def evaluar_modelo(df_test, df_train, models):
  X_train, y_train, X_test, y_test = split_dataset_train_test(df_train=df_train,
                                                              df_test=df_test)

  models_error=dict()
  models_fit=dict()
  models_predict=dict()
  for key,model in models.items():
      model.fit(X=X_train,y=y_train)
      # predicciones
      predict=model.predict(X_test)
      # error mae
      mae= metrics.mean_absolute_error(y_test, predict)
      # error cuadratico medio
      rmse= np.sqrt (metrics.mean_squared_error(y_test, predict)) 
      #  error cuadratico r2
      score= metrics.r2_score(y_test,predict) 
      # dict con errores del modelo
      models_error[key]={'MAE':round (mae,3), 'RMSE':round (rmse,3), 'SCORE':round(score,3)  }
      # valores de prediccion
      models_predict[key]=predict
      # sacamos el dict de modelos
      models_fit[key]=model
  return (models_fit, models_predict,models_error) 

In [13]:
def show_error_for_ca(y_pred_rfr, y_pred_xgbr, y_pred_gbr, y_pred_abr, dataset, test_ini, test_fin, df_ccaa, criterion='rmse'):
  # y_pred_rfr, y_pred_xgbr, y_pred_gbr, y_pred_abr -- valores de la prediccion del modelo 

    # df de test
    df = dataset[dataset['fecha'].isin(pd.date_range (test_ini,test_fin))]

    df['predict_XGBoost']=y_pred_xgbr
    df['predict_AdaBoost']=y_pred_abr
    df['predict_RandomForest']=y_pred_rfr
    df['predict_GradientBoosting']=y_pred_gbr

    best_model={}
    best_error=0
    for ca in df['cod_ine'].unique():
        df_ca = df[df['cod_ine']==ca]
        if criterion == 'mae':
            #XGBoost
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_XGBoost'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_XGBoost']))
            best_error=mae
            best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'cod_ine':ca, 'Modelo':'XGBoost', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }
            #AdaBoost
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_AdaBoost'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_XGBoost']))
            if mae<best_error:
                best_error=mae
                best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'ccaa_iso':ca, 'Modelo':'AdaBoost', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }
            #RandomForest
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_RandomForest'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_RandomForest']))
            if mae<best_error:
                best_error=mae
                best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'cod_ine':ca, 'Modelo':'RandomForest', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }
            #GradientBoosting
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_GradientBoosting'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_GradientBoosting']))
            if mae<best_error:
                best_error=mae
                best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'cod_ine':ca, 'Modelo':'GradientBoosting', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }
        else:
            #XGBoost
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_XGBoost'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_XGBoost']))
            score= metrics.r2_score(df_ca['num_casos'], df_ca['predict_XGBoost']) 
            #print ('XGBoost: MAE:', mae, 'RMSE:', rmse)
            best_error=rmse
            best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'cod_ine':ca, 'Modelo':'XGBoost', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }
            #AdaBoost
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_AdaBoost'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_AdaBoost']))
            score= metrics.r2_score(df_ca['num_casos'], df_ca['predict_AdaBoost']) 
            #print ('AdaBoost: MAE:', mae, 'RMSE:', rmse)
            if rmse<best_error:
                best_error=rmse
                best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'cod_ine':ca, 'Modelo':'AdaBoost', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }
            #RandomForest
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_RandomForest'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_RandomForest']))
            score= metrics.r2_score(df_ca['num_casos'], df_ca['predict_RandomForest'])
            #print ('RandomForest: MAE:', mae, 'RMSE:', rmse)
            if rmse<best_error:
                best_error=rmse
                best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'cod_ine':ca, 'Modelo':'RandomForest', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }
            #GradientBoosting
            mae= metrics.mean_absolute_error(df_ca['num_casos'], df_ca['predict_GradientBoosting'])
            rmse= np.sqrt (metrics.mean_squared_error(df_ca['num_casos'], df_ca['predict_GradientBoosting'])) 
            score= metrics.r2_score(df_ca['num_casos'], df_ca['predict_GradientBoosting'])
            #print ('GradientBoosting: MAE:', mae, 'RMSE:', rmse)
            if rmse<best_error:
                best_error=rmse
                best_model[ca]={'nombre_ccaa':df_ccaa['nombre_ccaa'][ca], 'cod_ine':ca, 'Modelo':'GradientBoosting', 'MAE':round (mae,3), 'RMSE':round (rmse,3) }

    best_model=pd.DataFrame (best_model).T
    display(best_model)
    return (best_model)

# Minería del dato

In [7]:
# carga de nuestros datos
df = upload_files()

KeyboardInterrupt: ignored

In [ ]:
df

In [ ]:
df_ccaa = upload_files(sep=";")

In [ ]:
df_ccaa

In [ ]:
df_ccaa.drop(columns=['Nombre_CCAA_mov', 'Nombre_CCAA_vac', 'ccaa_iso'], inplace=True)
df_ccaa.rename(columns={"Nombre_CCAA_covid": "nombre_ccaa", "cod_ISO-3166": "cod_iso_ccaa"}, inplace=True)
df_ccaa.set_index('cod_ine', inplace=True)
df_ccaa

Dividimos el df entre train y test. Como los datos 

In [ ]:
min_fecha=df.fecha.min()
max_fecha=df.fecha.max()
print (min_fecha, ':', max_fecha)

In [ ]:
df['fecha'] = pd.to_datetime(df['fecha'],format='%Y-%m-%d')
df.info()

In [ ]:
df.shape 

In [ ]:
train_ini = min_fecha
train_fin = '2021-08-31'
test_ini = '2021-09-01'
test_fin = max_fecha

In [ ]:
df.columns

In [ ]:
numeric_cols = ['fecha', 'num_casos', 'lug_ocio', 'lug_mercado',
       'lug_parque', 'lug_estacion', 'lug_trabajo', 'lug_residencial',
       'dosis_administradas', 'porcentaje_dosis_administradas',
       'personas_una_dosis', 'personas_pauta_completa',
       'porcentaje_pauta_completa', 'Fallecidos', 'Hospitalizados', 'UCI',
       'cod_ine', 'dia_sem', 'dia_year', 'dia_mes', 'is_weekend']

In [ ]:
df_numeric = df[numeric_cols]
df_numeric

In [ ]:
df_train=df_numeric[df_numeric['fecha'].isin (pd.date_range (train_ini,train_fin))]
df_test=df_numeric[df_numeric['fecha'].isin (pd.date_range (test_ini,test_fin))]

In [ ]:
df_test.shape

In [ ]:
df_train.shape

Calculamos el error cuadratico

In [ ]:
# definimos los modelos de árbol de decisiones
models=dict()
models['RF']= RandomForestRegressor(random_state=0, criterion='mae')
models['XGB']= XGBRegressor(eval_metric='mae',objective='reg:squarederror',random_state=0)
models['GB']= GradientBoostingRegressor(random_state=0,criterion='mae')
models['AB']= AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mae'), random_state=0)

In [ ]:
models_fit, models_predict, models_error = evaluar_modelo(df_test, df_train, models)

In [ ]:
models_fit

In [ ]:
df_models_prefict = pd.DataFrame(models_predict)
df_models_prefict

In [ ]:
df_models_error = pd.DataFrame(models_error).T
df_models_error

# Iteración I: 

Añadimos los datos de casos al dia D con los valores de D-n con n [1,7]

In [ ]:
df

In [ ]:
# copiamos el df

df1 = df[numeric_cols]
for i in range (1,11):
  col = f"casos_D-{i}"
  df1[col]=df1.groupby(["cod_ine"])["num_casos"].shift(i, fill_value=0)

df1.columns



In [ ]:
# comprobamos si hay nulos
df1.isnull().values.any()

In [ ]:
# comprobamos las correlaciones de los nuevos atributos
f, ax = plt.subplots(figsize=(30, 20))
corr = df1.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
cmap="GnBu"
plt.title('Correlación entre variables')
sns.heatmap(corr, mask=mask, vmax=1, center=0,
            square=True, linewidths=.5,fmt='.1f',cbar_kws={"shrink": 0.9},ax=ax,annot=True)

In [ ]:
# Creamos los df de trains y test para la iteración I

df_train_I=df1[df1['fecha'].isin (pd.date_range (train_ini,train_fin))]
df_test_I=df1[df1['fecha'].isin (pd.date_range (test_ini,test_fin))]

In [ ]:
# calculamos los errores de esta iteración
models_fit_I, models_predict_I, models_error_I = evaluar_modelo(df_test_I, df_train_I, models)
df_models_error_I = pd.DataFrame(models_error_I).T
df_models_error_I

In [ ]:
models_predict_I['XGB']

In [ ]:
best_models_I = show_error_for_ca(y_pred_rfr=models_predict_I['RF'], y_pred_xgbr=models_predict_I['XGB'], 
                  y_pred_gbr=models_predict_I['GB'], y_pred_abr=models_predict_I['AB'], dataset=df1, 
                  test_ini=test_ini, test_fin=test_fin, df_ccaa=df_ccaa, criterion='rmse')

In [ ]:
df1.to_csv("df_iter1.csv", index=False)
files.download("df_iter1.csv")

# Iteración II

In [ ]:
# carga de nuestros datos
df = upload_files()

In [ ]:
mov_cols = [ 'lug_ocio', 'lug_mercado', 'lug_parque',
       'lug_estacion', 'lug_trabajo', 'lug_residencial']

In [ ]:
df2_corr = df[df.columns]
for i in [7, 14, 21, 28]:
  for c in mov_cols:
    col = f"{c}_D-{i}"
    df2_corr[col] = df2_corr.groupby(['cod_ine'])[c].shift(i)

df2_corr.fillna(0, inplace=True)

In [ ]:
df2_corr

In [ ]:
# calculamos la correlación de las variables
# comprobamos las correlaciones de los nuevos atributos
f, ax = plt.subplots(figsize=(50, 50))
corr = df2_corr.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
cmap="GnBu"
plt.title('Correlación entre variables')
sns.heatmap(corr, mask=mask, vmax=1, center=0,
            square=True, linewidths=.5,fmt='.1f',cbar_kws={"shrink": 0.9},ax=ax,annot=True)

In [ ]:
df2 = df1[df1.columns]
for i in [7, 14, 21, 28]:
  for c in mov_cols:
    col = f"{c}_D-{i}"
    df2[col] = df2.groupby(['cod_ine'])[c].shift(i)

df2.fillna(0, inplace=True)
df2.columns

In [ ]:
df2.to_csv("df_iter2.csv", index=False)
files.download("df_iter2.csv")

In [ ]:
df2 =  upload_files()
df2['fecha'] = pd.to_datetime(df2['fecha'],format='%Y-%m-%d')

In [ ]:
# definimos los modelos de árbol de decisiones
models=dict()
models['RF']= RandomForestRegressor(random_state=0, criterion='mae')
models['XGB']= XGBRegressor(eval_metric='mae',objective='reg:squarederror',random_state=0)
models['GB']= GradientBoostingRegressor(random_state=0,criterion='mae')
models['AB']= AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mae'), random_state=0)

In [ ]:
# Creamos los df de trains y test para la iteración I

df_train_II=df2[df2['fecha'].isin (pd.date_range (train_ini,train_fin))]
df_test_II=df2[df2['fecha'].isin (pd.date_range (test_ini,test_fin))]

# calculamos los errores de esta iteración
models_fit_II, models_predict_II, models_error_II = evaluar_modelo(df_test_II, df_train_II, models)

In [ ]:
df_train_II

In [ ]:
df_models_error_II = pd.DataFrame(models_error_II).T
df_models_error_II

In [ ]:
best_models_II = show_error_for_ca(y_pred_rfr=models_predict_II['RF'], y_pred_xgbr=models_predict_II['XGB'], 
                  y_pred_gbr=models_predict_II['GB'], y_pred_abr=models_predict_II['AB'], dataset=df2, 
                  test_ini=test_ini, test_fin=test_fin, df_ccaa=df_ccaa, criterion='rmse')

# optimización de hiperparámetros

In [14]:
df1 = upload_files() 

Saving df_iter1.csv to df_iter1.csv
User uploaded file "df_iter1.csv" with length 1959541 bytes


In [18]:
df1['fecha'] = pd.to_datetime(df1['fecha'],format='%Y-%m-%d')
min_fecha=df1.fecha.min()
max_fecha=df1.fecha.max()
train_ini = min_fecha
train_fin = '2021-08-31'
test_ini = '2021-09-01'
test_fin = max_fecha
df_train=df1[df1['fecha'].isin (pd.date_range (train_ini,train_fin))]
df_test=df1[df1['fecha'].isin (pd.date_range (test_ini,test_fin))]

In [15]:
def get_errors (real,predict, name_col=''):
    #bias = np.mean(real - predict)
    mae= metrics.mean_absolute_error(real, predict)
    #mse=metrics.mean_squared_error(real, predict)
    rmse= np.sqrt (metrics.mean_squared_error(real, predict))   
    #mape = np.mean(np.abs((real - predict) / real) * 100)   
    score= metrics.r2_score(real,predict) 
    
    df = pd.DataFrame([mae, rmse, score], index=['MAE','RMSE','SCORE'], columns=[name_col])
    return df.T

In [16]:
# Funciones 
def grid_search(df_train, df_test, model, param, title, show_features_importance=True):
    X_train, y_train, X_test, y_test =split_dataset_train_test(df_train, df_test)
    tscv = TimeSeriesSplit(n_splits=3)
    grid = GridSearchCV(estimator = model, param_grid=param, cv=tscv, n_jobs=-1, scoring='neg_mean_squared_error')
    grid.fit(X = X_train,y=y_train)
    #report(grid.cv_results_)
    best_params = grid.best_params_
    for i in best_params:
        print (i, ":", best_params[i])
    fit_model= grid.best_estimator_
    y_pred=fit_model.predict (X_test)
    error_test=get_errors(y_test, y_pred,title)
    display (error_test)
    return (fit_model, y_pred)


def random_search(df_train, df_test, model, param, title, show_features_importance=True):

    X_train, y_train, X_test, y_test =split_dataset_train_test(df_train, df_test)
    tscv = TimeSeriesSplit(n_splits=3)
    random= RandomizedSearchCV(estimator = model, param_distributions = param, n_iter = 50, scoring='neg_mean_squared_error',cv = tscv, random_state=0, n_jobs = -1)
    random.fit(X = X_train,y=y_train)
    #report(grid.cv_results_, n_top = 5)
    best_params = random.best_params_
    for i in best_params:
        print (i, ":", best_params[i])
    fit_model= random.best_estimator_
    y_pred=fit_model.predict (X_test)
    error_test=get_errors(y_test, y_pred,title)
    display (error_test)
    return (fit_model, y_pred)

In [19]:
hyperparam = {"n_estimators": [16, 64, 128, 256, 512], 
              "max_depth": [4,6,8,10], 
              "min_samples_split": [2, 4, 6, 8], 
              "min_samples_leaf": [10, 12,16,18,20], 
              "bootstrap": [True, False] 
             }
model = RandomForestRegressor(criterion='mse', random_state=0)
title = 'RandomForest'
rfr_tune_grid,predict_rfr_tune_grid= grid_search(df_train=df_train, 
                                                  df_test=df_test, model=model, 
                                                  param=hyperparam, 
                                                  title='RandomForest')

KeyboardInterrupt: ignored

In [ ]:
# este
hyperparam = {'n_estimators': [100,200,300,400,500],
              'max_depth':range(1,10),
              'min_child_weight':range(1,5),
              'subsample': [i / 10.0 for i in range(6, 9)],
              'colsample_bytree': [i / 10.0 for i in range(6, 11)],   
              'eta': [.3, .2, .1, .05, .01, .005],
              'gamma':[i/10.0 for i in range(0,6)]   
             }
model = XGBRegressor(eval_metric='rmse',objective='reg:squarederror', random_state=0)
title = 'XGBoost'
xgb_grid_search,predict_xbg_grid_search = grid_search(df_train=df_train, 
                                                  df_test=df_test, model=model, 
                                                  param=hyperparam, 
                                                  title=title)

In [ ]:
# este
hyperparam = {
 "n_estimators": [16, 32, 64, 100, 128, 200, 256],
 "learning_rate" : [0.01, 0.05, 0.75, 0.1, 0.15, 0.2, 0.25,0.5],
 "max_features" : ["auto", "sqrt"],
 "min_samples_split": [2, 4, 6, 8, 10],
 'max_depth': range(3,10,2),
 "loss" : ['ls', 'lad'],
 'warm_start':[True, False]
 }
model = GradientBoostingRegressor(criterion='mse', random_state=0)
title = 'GradientBoosting'

gbr_grid_search,predict_grid_random_search = grid_search(df_train=df_train, 
                                                  df_test=df_test, model=model, 
                                                  param=hyperparam, 
                                                  title=title)

In [ ]:
# este
hyperparam = {
 "n_estimators": [8, 16, 32, 64, 70, 80, 90, 100, 128],
 "learning_rate" : [0.005, 0.01, 0.015, 0.02, 0.025, 0.05, 0.1],
 "loss" : ['linear', 'square', 'exponential']
 }

model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse'),random_state=0)
title = 'AdaBoost'

abr_grid_search,predict_grid_random_search = grid_search(df_train=df_train, 
                                                  df_test=df_test, model=model, 
                                                  param=hyperparam, 
                                                  title=title)